## LangChain Chat Memory

- ConversationBufferMemory 
    - It's like a list with all the messages
- ConversationBufferWindowMemory
    - Quite similar to the above one, but returns just only a chunk of the list
- ConversationSummaryMemory
    - Instead of storing the complete messages, it stores only a summary of the conversation
- ConversationSummaryBufferMemory
    - Combination between ConversationSummaryMemory and ConversationBufferWindowMemory
    You get a summary of the conversation + Xk of recent tokens stored

In [1]:
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

### ConversationSummaryMemory - DEPRECATED WAY

In [3]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain

In [4]:
memory = ConversationSummaryMemory(llm=llm)

C:\Users\luis\AppData\Local\Temp\ipykernel_16472\1952740995.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [5]:
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

C:\Users\luis\AppData\Local\Temp\ipykernel_16472\3822530700.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [6]:
chain.invoke({"input": "hello there my name is James"})
chain.invoke({"input": "I am researching the different types of conversational memory."})
chain.invoke({"input": "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory."})
chain.invoke({"input": "Buffer memory just stores the entire conversation"})
chain.invoke({"input": "Buffer window memory stores the last k messages, dropping the rest."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello there my name is James
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as James. The AI greets James and expresses its willingness to chat and help, inquiring about how his day is going.
Human: I am researching the different types of conversational memory.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after

{'input': 'Buffer window memory stores the last k messages, dropping the rest.',
 'history': 'The human introduces himself as James, and the AI greets him, expressing its willingness to chat and asking about his day. James mentions he is researching different types of conversational memory. The AI explains that conversational memory includes short-term memory (retaining context within a single conversation), long-term memory (retaining information across multiple interactions), contextual memory (remembering the context and emotional tone of conversations), and dynamic memory (adapting based on new information). The AI then asks if James is looking into a specific type of conversational memory or exploring the topic more broadly. James shares that he has been looking at ConversationBufferMemory and ConversationBufferWindowMemory. The AI describes ConversationBufferMemory as tracking recent interactions to maintain context in short conversations, while ConversationBufferWindowMemory use

In [7]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as James, and the AI greets him, expressing its willingness to chat and asking about his day. James mentions he is researching different types of conversational memory. The AI explains that conversational memory includes short-term memory (retaining context within a single conversation), long-term memory (retaining information across multiple interactions), contextual memory (remembering the context and emotional tone of conversations), and dynamic memory (adapting based on new information). The AI then asks if James is looking into a specific type of conversational memory or exploring the topic more broadly. James shares that he has been l

{'input': 'What is my name again?',
 'history': 'The human introduces himself as James, and the AI greets him, expressing its willingness to chat and asking about his day. James mentions he is researching different types of conversational memory. The AI explains that conversational memory includes short-term memory (retaining context within a single conversation), long-term memory (retaining information across multiple interactions), contextual memory (remembering the context and emotional tone of conversations), and dynamic memory (adapting based on new information). The AI then asks if James is looking into a specific type of conversational memory or exploring the topic more broadly. James shares that he has been looking at ConversationBufferMemory and ConversationBufferWindowMemory. The AI describes ConversationBufferMemory as tracking recent interactions to maintain context in short conversations, while ConversationBufferWindowMemory uses a sliding window to dynamically adjust the 

### ConversationSummaryMemory - Newest way

In [8]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

In [9]:
system_prompt = "You are a helpful assistant called Zeta."

In [10]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [11]:
pipeline = prompt_template | llm

In [12]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, SystemMessage

In [13]:
class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)

    def __init__(self, llm: ChatOpenAI):
        super().__init__(llm=llm)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """
        Create a summary
        """
        self.messages.extend(messages)
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much as possible relevant information as possible"
            ),
            HumanMessagePromptTemplate.from_template([
                "Existing conversation summary: \n{existing_summary}\n\n"
                "New messages: \n{messages}"
            ])
        ])

        #format the messages and invoke the llm
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=self.messages,
                messages=[x.content for x in messages] 
            )
        )

        # replace the existing history with a single system summary message
        self.messages = [SystemMessage(content=new_summary.content)]

    def clear(self) -> None:
        """
        Clear the history
        """
        self.messages = []

In [14]:
chat_map = {}
def get_chat_history(session_id: str, llm: ChatOpenAI) -> ConversationSummaryMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exists, create a new chat history
        chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    # return the chat history
    return chat_map[session_id]

In [15]:
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory

In [16]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default"
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm
        )
    ]
)

In [17]:
pipeline_with_history.invoke(
    {"query": "Hy, my name is Luis"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content='Hi Luis! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 26, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C26auRE5o1HFTfKYFw9sDBdeuOwr8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bf146b6f-92d9-4eca-9054-17788dc93582-0', usage_metadata={'input_tokens': 26, 'output_tokens': 10, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
chat_map["id_123"].messages

[SystemMessage(content='In the conversation, Luis introduces himself by saying, "Hy, my name is Luis." The AI responds warmly, asking, "Hi Luis! How can I assist you today?"', additional_kwargs={}, response_metadata={})]

In [19]:
pipeline_with_history.invoke(
    {"query": "What's my name again?"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content='Your name is Luis! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 65, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C26aw2smBaYwwIbWNWDgLy0Wu4nhq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a5079f89-1e5b-4d0b-9178-a9f23d5886b8-0', usage_metadata={'input_tokens': 65, 'output_tokens': 12, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [20]:
chat_map["id_123"].messages

[SystemMessage(content='In the conversation, Luis introduces himself by saying, "Hy, my name is Luis." The AI responds warmly, asking, "Hi Luis! How can I assist you today?" Luis then asks, "What\'s my name again?" to which the AI replies, "Your name is Luis! How can I assist you today?"', additional_kwargs={}, response_metadata={})]